In [ ]:
# プロジェクト直下で notebook を開いている想定:
# project/
# ├─ relicot/
# │   ├─ CoT/
# │   └─ note/test.ipynb  ← これ
import sys, pathlib
root = pathlib.Path(__file__).resolve().parents[1] if "__file__" in globals() else pathlib.Path.cwd().parents[0]
sys.path.insert(0, str(root))  # プロジェクト直下を import パスに追加
print("PYTHONPATH added:", root)

In [14]:
from relicot.CoT.cot_module import CoTGenerator
from relicot.CoT import cot_prompts

# まずはダミー生成関数で疎結合インターフェースを確認
def dummy_generate(prompt: str, max_new_tokens=256, temperature=0.7, **_):
    return (
        "Step 1: 条件を整理する。\n"
        "Step 2: 計算手順を適用する。\n"
        "Answer: 42"
    )

gen = CoTGenerator(generate_fn=dummy_generate)

problem = "2と40を足すと？"
res = gen.run(problem=problem, template="cot_default_ja")

print("Steps:", res["steps"])
print("Answer:", res["answer"])


Steps: ['条件を整理する。', '計算手順を適用する。']
Answer: 42


In [15]:
# !pip install transformers accelerate torch --quiet

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"  # 好きな指示追従モデルに変更可
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)

def hf_generate(prompt: str, max_new_tokens=256, temperature=0.7, top_p=0.95, stop=None):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            eos_token_id=tokenizer.eos_token_id,
        )
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # 出力からプロンプト部分を差し引く（モデルによっては不要）
    if text.startswith(prompt):
        text = text[len(prompt):].lstrip()
    # stop語があればトリム
    if stop:
        for s in ([stop] if isinstance(stop, str) else stop):
            idx = text.find(s)
            if idx >= 0:
                text = text[:idx]
                break
    return text

from relicot.CoT.cot_module import CoTGenerator
gen = CoTGenerator(generate_fn=hf_generate, max_input_tokens=4096)

problem = "10個のリンゴを3人でできるだけ均等に分けると、一人あたりいくつ？"
res = gen.run(problem=problem, template="cot_default_ja", tokenizer=tokenizer)
print(res["steps"])
print("Answer:", res["answer"])


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

['まず、10個のリンゴを3人でできるだけ均等に分ける場合を考えます。これは10を3で割ることと同じです。', '10 ÷ 3 を計算すると、商は3で、余りは1になります。つまり、一人あたり3つのリンゴを得ることができます。', 'しかし、10個のリンゴを完全に均等に分けることはできません。残りの1個は3人の誰かに追加して分ける必要があります。']
Answer: 一人あたり3個のリンゴと、一部のリンゴが分配できませんでした。または、一人あたり約3.33個（小数点以下を四捨五入せずに表記）。ただし、実際には個数を均等に分配することは不可能です。 Answer: 一人あたり3個のリンゴと1個の余り。または、一人あたり約3.33個（小数点以下を四捨五入せずに表記）。ただし、実際には個数を均等に分配することは不可能です。 最終的な結論として、一人当たりのリンゴの


In [16]:
sc = gen.run_self_consistency(
    problem="7×8の答えは？",
    template="cot_sc_ja",
    k=5,  # サンプル数
)
print("Majority:", sc["answer_majority"])
print("All answers:", sc["answers"])


Majority: 56
All answers: ['56', '56', '56', '56', '56']


In [17]:
res_json = gen.run_json(
    problem="3人で12個の飴を均等に分けたら一人あたり？",
    template="cot_json_ja",
    strict=False  # TrueだとJSON失敗時に空返し、Falseなら自由文パースにフォールバック
)
print(res_json["steps"])
print("Answer:", res_json["answer"])


['12個の飴を3人で均等に分けるためには、12を3で割る', '12 ÷ 3 = 4', '一人あたり4個の飴が得られる']
Answer: 4
